1. 파이토치 제공 데이터 사용
2. 같은 클래스 별 폴더 이미지 데이터 이용
3. 개인 데이터 사용(2 types)

In [2]:
import torch
import torchvision
import torchvision.transforms as tr # 데이터를 불러오면서 전처리를 바로 할 수 있게 불러와주는 라이브러리
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [3]:
transf = tr.Compose([tr.Resize(8),tr.ToTensor()])
# 전처리 작업을 Compose 안에 일렬로 해준다.
# compose = 순서대로 작업해준다.
# PIL 이미지 형태여야지 사용할 수 있다!!!!!

In [4]:
trainset = torchvision.datasets.CIFAR10(root='A:\chch\chchdata\data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='A:\chch\chchdata\data', train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainset[0][0].size()
# cifar10이 rgb 채널이라 3, 8, 8이 된다.
# 파이토치는 채널 수가 앞에 위치한다.

torch.Size([3, 8, 8])

In [6]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)
# num_workers =  데이터로드를 할 때 서브 프로세스를 몇 개를 쓸 것인지 설정 -> 파라미터 튜닝에 해당
# 고려 사항 : 학습 환경의 GPU개수, CPU개수, I/O 속도, 메모리 등

In [7]:
len(trainloader)

1000

In [8]:
dataiter = iter(trainloader) # 데이터를 하나씩 불러온다는 것
images, labels = dataiter.next()

images.size()
#torch.Size([50, 3, 8, 8]) -> 배치사이즈, 채널 수 , 이미지 너비, 이미지 높이

torch.Size([50, 3, 8, 8])

2. 같은 클래스 별 폴더 이미지 데이터 이용

In [9]:
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='A:\\chch\\chchdata\\data\\gender', transform=transf) 
# './class' 안에 있는 이미지를 자동으로 알아서 찾아주고, 각각의 다른 폴더에 대해 라벨링까지 해준다. transform을 설정해 전처리도 바로 할 수 있다.
trainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)
print(len(trainloader))

2


In [10]:
trainset[0][0].size()

torch.Size([3, 16, 16])

3. 개인 데이터 사용(2 types)

In [11]:
# import preprocessing
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

# preprocessing..
# train_images, train_labels = preprocessing(train_images, train_labels)
print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [12]:
class TensorData(Dataset):
    def __init__(self, x_data, y_data): # 외부데이터를 받기 위해 x_data, y_data 추가
        self.x_data = torch.FloatTensor(x_data) # 숫자 속성 정하는 tensor
        self.x_data = self.x_data.permute(0,3,1,2) 
        # 이미지 개수, 채널 수 , 이미지 너비, 이미지 높이 -> 순서를 바꾸기 위해 permute 사용
        # (20, 32, 32, 3) -> (20, 3, 32, 32)
        self.y_data = torch.LongTensor(y_data)  # 숫자 속성 정하는 tensor
        self.len = self.y_data.shape[0] # 데이터의 개수 산출하기

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] # train_data[0][0] 
    def __len__(self):
        return self.len

In [13]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [14]:
train_data[0][0].size() # torch.Size([3, 32, 32]) , x_data만 불러오는 것
# train_data[0][1].size() #torch.Size([1]) -> y 라벨 값을 불러오는데 0 또는 1 이니까 사이즈는 1
# train_data[3][1].size() #torch.Size([1])

torch.Size([3, 32, 32])

In [15]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

images.size()

torch.Size([10, 3, 32, 32])

4. 개인 데이터 사용하는 이유
- 데이터가 다른 작업도 쓰는 경우에는 dir 함부로 못바꾼다. 파이토치에서 제공하는 transform 의 종류가 비교적 제한적이다. 
- 디테일한 전처리 작업은 파이토치에서 사용하지 않는다. 

개인 데이터에 transform 사용하기

In [16]:
'''
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
    
    def __getitem__(self, index):
    
    def __len__(self):
'''
# 위의 코드를 잘 활용하면 좋다. 

class MyDataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    def __getitem__(self, index): # getitem은 튜플형태로 데이터를 내보내준다.
        sample = self.x_data[index], self.y_data[index]
        if self.transform: # 전처리를 해서 내보낸다.
            sample = self.transform(sample)
        return sample
    def __len__(self):
        return self.len

class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        return inputs, torch.LongTensor(labels)

class LinearTensor:
    def __init__(self, slope=1, bias=0): 
        self.slope = slope
        self.bias = bias

    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope*inputs + self.bias #원하는 계산 기입
        return inputs, labels



In [17]:
trans = tr.Compose([ToTensor(),LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

In [18]:
first_data = ds1[0]
features, labes = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [20]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

# images1

위에는 numpy형태라 tr.Compose를 사용하면 에러가 난다. 
tr을 꼭 사용하고 싶다면 아래 방식을 이용.